# Пайплайн

Пдф-ки:

In [ ]:
%%capture
!pip install img2table[easyocr]

In [ ]:
from img2table.document import PDF
from img2table.ocr import EasyOCR
import pandas as pd
import requests

easyocr = EasyOCR(lang=["ru"], kw={"gpu": True})

In [ ]:
import os

print('[')
for file in os.listdir():
  if '.pdf' in file:
    subject = file.split('_')[0].strip().lower()
    if subject == 'искусство':
        subject = 'мхк'
    elif subject == 'физическая':
        subject = 'физра'
    print("('" + subject + "', '" + file + "'),")
print(']')

In [ ]:
filenames = [
('право', 'Право_1.pdf'),
('физра', 'Физическая_культура_(девушки)_1.pdf'),
('экология', 'Экология_1.pdf'),
('технология', 'Технология_(д)_2.pdf'),
]

In [ ]:
import pandas as pd
import numpy as np
import re


def fill_none_cells_above_first_value(column):
    first_value_index = column.first_valid_index()
    if first_value_index is not None:
        first_value = column[first_value_index]
        mask = (column.isna() & (column.index < first_value_index)).values
        column[mask] = first_value
    return column


def find_specific_column(sheet_df, name):
    for col in sheet_df.columns:
        if re.search(name, re.sub(r'[^a-zа-яё ]', '', str(col).lower())):
            return col
    return None


def parse_df(df, subject='предмет', class_='9'):
    df = df[~df.apply(lambda row: row.astype(str).str.contains('регионального')).any(axis=1)]
    df = df.apply(fill_none_cells_above_first_value, axis=0)
    df.columns = df.iloc[0]
    df = df.loc[:, ~df.columns.duplicated()]
    for col in df.columns:
        df = df[df[col] != col]

    class_column = find_specific_column(df, 'класс')
    if class_column is None:
        df['класс'] = class_
    else:
        df.rename(columns={class_column: 'класс'}, inplace=True)

    status_column = find_specific_column(df, 'статус')
    if status_column is None:
        status_column = find_specific_column(df, 'диплом')
    if status_column is None:
        status_column = find_specific_column(df, 'рейтинг')
    df.rename(columns={status_column: 'результат'}, inplace=True)
    try:
        df['результат'] = df['результат'].fillna('Участник')
    except KeyError:
        df['результат'] = 'Участник'

    fio_column = find_specific_column(df, 'фио')
    if fio_column is None:
        fio_column = find_specific_column(df, 'фамилия имя')
        if fio_column is None:
            fio_column1 = find_specific_column(df, 'фамилия')
            fio_column2 = find_specific_column(df, 'имя')
            df['фио'] = df[fio_column1] + ' ' + df[fio_column2]
            fio_column3 = find_specific_column(df, 'отчество')
            if fio_column3 is not None:
                df['фио'] = df['фио'] + ' ' + df[fio_column3]
        else:
            df.rename(columns={fio_column: 'фио'}, inplace=True)
    else:
        df.rename(columns={fio_column: 'фио'}, inplace=True)

    df['предмет'] = subject
    df = df[[col for col in ['предмет', 'класс', 'фио', 'результат'] if col in df.columns]]

    df = df.replace(r'\n',' ', regex=True)
    df['фио'] = df['фио'].replace(r'[^А-Яа-яёЁ -]',' ', regex=True)
    df['класс'] = df['класс'].replace(r'(II)|И', '11', regex=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    return df


all_tables = []
for subject, filename in filenames:
    pdf = PDF(filename,
              detect_rotation=True,
              pdf_text_extraction=True)
    extracted_tables = pdf.extract_tables(ocr=easyocr, min_confidence=30)

    # таблички в пандас
    pd_tables = []
    for page, tables in extracted_tables.items():
        for idx, table in enumerate(tables):
            pd_tables.append(table.df)

    match_ = re.search(r'\d+', filename)
    class_ = match_.group() if match_ else None
    if class_ in ['1', '2', '3']:
        class_ = str(int(class_) + 8)
    overall_table = parse_df(pd.concat(pd_tables, ignore_index=True), subject, class_)
    all_tables.append(overall_table)
    display(overall_table)

final_df = pd.concat(all_tables, ignore_index=True)
final_df.to_csv('final_df.csv', index=False)

Эксельки:

In [ ]:
%%capture
!pip install xls2xlsx

In [ ]:
subject_filename = [
 ("",'камчатка15-16.xlsx'),
]

In [ ]:
import openpyxl
import pandas as pd
import re
from xls2xlsx import XLS2XLSX


def convert_xls_to_xlsx(filename):
    x2x = XLS2XLSX(filename)
    x2x.to_xlsx(filename + "x")


def fill_status_nan(sheet, keyword):
    found_status = False

    for row_idx, row in sheet.iterrows():
        for col_idx, cell_value in row.items():
            if keyword in str(cell_value).lower():
                found_status = True
                sheet.at[row_idx, col_idx] = keyword
                current_row = row_idx
                while current_row > 0 and pd.isna(sheet.at[current_row - 1, col_idx]):
                    sheet.at[current_row - 1, col_idx] = keyword
                    current_row -= 1
                current_row = row_idx
                while current_row < sheet.shape[0] - 1 and pd.isna(sheet.at[current_row + 1, col_idx]):
                    sheet.at[current_row + 1, col_idx] = keyword
                    current_row += 1


def find_header_row(sheet_df):
    for i, row in sheet_df.iterrows():
        row_lower = row.astype(str).str.lower()
        if 'фамилия' in row_lower.values or 'фио' in row_lower.values:
            return i
    return None


def find_specific_column(sheet_df, name):
    for col in sheet_df.columns:
        if re.search(name, str(col).lower()):
            return col
    return None


def parse_excel_file(file_path, subject='предмет'):
    xls = pd.ExcelFile(file_path)

    if file_path.split('.')[-1] == 'xlsx':
        xls = pd.ExcelFile(file_path)
    else:
        convert_xls_to_xlsx(file_path)
        file_path = file_path + 'x'
        xls = pd.ExcelFile(file_path)

    parsed_data = pd.DataFrame(columns=['предмет', 'класс', 'фио', 'результат'])

    for sheet_name in xls.sheet_names:
        sheet_df = pd.read_excel(file_path, sheet_name, engine='openpyxl', header=None)
        fill_status_nan(sheet_df, 'статус')
        fill_status_nan(sheet_df, 'диплом')
        header_row_index = find_header_row(sheet_df)

        if header_row_index is not None:
            sheet_df.columns = sheet_df.iloc[header_row_index]
            sheet_df = sheet_df.iloc[header_row_index+1:]

            class_column = find_specific_column(sheet_df, 'класс')
            if class_column is None:
                sheet_df['класс'] = re.sub(r'[^0-9]','', sheet_name)
            else:
                sheet_df.rename(columns={class_column: 'класс'}, inplace=True)

            try:
                sheet_df.drop(columns='результат', inplace=True)
            except KeyError:
                pass

            status_column = find_specific_column(sheet_df, 'статус')
            if status_column is None:
                status_column = find_specific_column(sheet_df, 'диплом')
            if status_column is None:
                status_column = find_specific_column(sheet_df, 'рейтинг')

            sheet_df.rename(columns={status_column: 'результат'}, inplace=True)
            sheet_df['результат'] = sheet_df['результат'].fillna('Участник')

            fio_column = find_specific_column(sheet_df, 'фио')
            if fio_column is None:
                fio_column = find_specific_column(sheet_df, 'фамилия имя')
                if fio_column is None:
                    fio_column1 = find_specific_column(sheet_df, 'фамилия')
                    fio_column2 = find_specific_column(sheet_df, 'имя')
                    sheet_df['фио'] = sheet_df[fio_column1] + ' ' + sheet_df[fio_column2]
                    fio_column3 = find_specific_column(sheet_df, 'отчество')
                    if fio_column3 is not None:
                        sheet_df['фио'] = sheet_df['фио'] + ' ' + sheet_df[fio_column3]
                else:
                    sheet_df.rename(columns={fio_column: 'фио'}, inplace=True)
            else:
                sheet_df.rename(columns={fio_column: 'фио'}, inplace=True)

            sheet_df = sheet_df[[col for col in ['класс', 'фио', 'результат'] if col in sheet_df.columns]]
            sheet_df.dropna(inplace=True)
            parsed_data = pd.concat([parsed_data, sheet_df], ignore_index=True)

    parsed_data['предмет'] = subject
    return parsed_data


all_dfs = []
for subject, filename in subject_filename:
    result_df = parse_excel_file(filename, subject)
    display(result_df)
    all_dfs.append(result_df)
pd.concat(all_dfs).to_csv('finally.csv', index=False)

# Примеры парсинга

In [ ]:
!sudo apt install tesseract-ocr-rus

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd tesseract-ocr-rus
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 6,088 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-rus all 1:4.00~git30-7274cfa-1.1 [1,271 kB]
Fetched 6,088 kB in 1s (6,731 kB/s)
debconf: unable to i

In [ ]:
! pip install pdfplumber img2table img2table[easyocr] python-docx wget func_timeout translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import wget
import requests
from bs4 import BeautifulSoup

from time import sleep
from tqdm import tqdm
from func_timeout import func_timeout
from translate import Translator

import pdfplumber
from docx import Document

from img2table.document import Image
from img2table.document import PDF
from img2table.ocr import TesseractOCR
#from img2table.ocr import EasyOCR
#from img2table.ocr import PaddleOCR

In [ ]:
subj_codes = {'экономике': "Экономика", "экологии": "Экология",
              "химии": "Химия", "французскому": "Французский язык",
              "физической": "Физическая культура", "физике": "Физика",
              "технологии": "Технология", "русскому": "Русский язык",
              "праву": "Право", "обществознанию": "Обществознание",
              "ОБЖ": "ОБЖ", "немецкому": "Немецкий язык",
              "математике": "Математика", "литературе": "Литература",
              "истории": "Истории", "искусству": "Искусство",
              "информатике": "Информатика", "географии": "География",
              "биологии": "Биология", "астрономии": "Астрономия",
              "английскому": "Английский язык"}

In [ ]:
def extract_table_pdf(filename):
  pdf = PDF(src=filename)
  ocr = TesseractOCR(n_threads=1,
                     lang="rus",
                     psm=11)

  pdf_tables = pdf.extract_tables(ocr=ocr)
  return pd.concat([table[0].df if table else pd.DataFrame() for table in pdf_tables.values()])


def extract_table_docx(filename):
  document = Document(filename)
  dfs = []
  for tb in document.tables:
    data = [[cell.text for cell in row.cells] for row in tb.rows]
    df = pd.DataFrame(data)
    dfs.append(df)
  return pd.concat(dfs)


def save_csv(tables, region, filename):
  finals = pd.concat(tables)
  finals['уровень_олимпиады'] = "региональный"
  finals['административная_единица'] = region
  finals = finals[['административная_единица', 'год', 'уровень_олимпиады', 'предмет', 'класс', 'ФИО', 'статус']]
  finals.to_csv(filename)

### Ленинградская область

In [ ]:
links = {2023: "http://center-intellect.ru/vsosh/rezultaty-region/",
         2022: "http://center-intellect.ru/vsosh/RegionalStage/protocolRS/2022/",
         2021: "http://center-intellect.ru/vsosh/RegionalStage/protocolRS/index2021.php",
         2020: "http://center-intellect.ru/vsosh/RegionalStage/protocolRS/",
         2019: "http://center-intellect.ru/vsosh/rezultaty-region/rezultaty-region-2019.php",
         2018: "http://center-intellect.ru/vsosh/rezultaty-region/rezultaty-region-2018.php"}


data = []
for year, href in links.items():
  r = requests.get(href)
  page = BeautifulSoup(r.text, 'html.parser')
  table = page.find('table')
  table_body = table.find('tbody')
  rows = table_body.find_all('tr')
  subjs = []
  hrefs = []
  for row in rows:
    cols = row.find_all('td')
    if len(cols) < 2:
      continue
    all_a = [(col.get('onclick').split("'")[1] if col.get('onclick') else row.get('onclick').split("'")[1])
    if not col.find('a') else col.find('a').get('href') for col in cols[1:]]
    all_a = ["http://center-intellect.ru" + a if "http" not in a else a for a in all_a]
    all_a = [a.replace("https", "http") for a in all_a]
    if not all_a:
      continue
    subjs.extend(["Технология" if "технология" in cols[0].text.lower() else cols[0].text.split('-')[-1].strip().capitalize()] * len(all_a))
    hrefs.extend(all_a)
  data.append(pd.DataFrame({'year': year, 'subj': subjs, 'href': hrefs}))


doc_links = pd.concat(data)

In [ ]:
doc_links

,year,subj,href
0,2023,Французский язык,http://center-intellect.ru/vsosh/rezultaty-reg...
1,2023,Русский язык,http://center-intellect.ru/upload/files/docs/v...
2,2023,Русский язык,http://center-intellect.ru/upload/files/docs/v...
3,2023,Русский язык,http://center-intellect.ru/upload/files/docs/v...
4,2023,Литература,http://center-intellect.ru/upload/files/docs/v...
...,...,...,...
47,2018,Технологии \n\t\t (номинация «техника и технич...,http://center-intellect.ru/vsosh/rezultaty-reg...
48,2018,Технологии \n\t\t (номинация «техника и технич...,http://center-intellect.ru/vsosh/rezultaty-reg...
49,2018,Основы безопасности жизнедеятельности:,http://center-intellect.ru/vsosh/rezultaty-reg...
50,2018,Основы безопасности жизнедеятельности:,http://center-intellect.ru/vsosh/rezultaty-reg...


In [ ]:
TABLES = []
k = -1
for index, row in doc_links.iterrows():
    k += 1
    if k == 253:
      continue
    name = row.href.split("/")[-1]

    f = requests.get(row.href)
    with open(name, mode="wb") as file:
      file.write(f.content)

    table = extract_table_pdf(name)
    if not table.iloc[0, 0]:
      continue
    table.columns = table.iloc[0].fillna('Ambigous')
    table.drop(0, inplace=True)
    names = table.loc[:, ['имя' in col.lower() or 'фио' in col.lower() for col in table.columns]]
    names = names.iloc[:, 0].apply(lambda x: x.replace("\n", " ") if isinstance(x, str) else None).values if not names.empty else None
    classes = table.loc[:, ['класс' in col.lower() for col in table.columns]]
    classes = classes.iloc[:, 0].values if not classes.empty else None
    statuses = table.iloc[:, -1].values
    statuses = ['участник' if stat is None or stat is np.nan else stat.lower() for stat in statuses]

    TABLES.append(pd.DataFrame({"год": row.year, "предмет": row.subj, "ФИО": names, "класс": classes, "статус": statuses}))


save_csv(TABLES, "Ленинградская область", "len_obl.csv")

### г. Москва

In [ ]:
data = []

# список ссылок
for year in unilist_links:
  r = requests.get(f"https://vos.olimpiada.ru/{year}/region/results")
  page = BeautifulSoup(r.text, 'html.parser')
  table = page.find_all('ul')[0]
  all_a = table.find_all('a')

  subj = [a.text.split('.')[-1].strip().capitalize() for a in all_a]
  subj = ['Технология' if (s in tech_names) or ('Профиль' in s) else s for s in subj]
  hrefs = [a.get('href') for a in all_a]
  data.append(pd.DataFrame({'year': year, 'subj': subj, 'href': hrefs}))


# вложенный список ссылок
for year in multilist_links:
  r = requests.get(f"https://vos.olimpiada.ru/{year}/region/results")
  page = BeautifulSoup(r.text, 'html.parser')
  tables = page.find_all('ul')[1:]
  all_a = []
  for table in tables:
    all_a.extend(table.find_all('a'))

  subj = [a.text.split('.')[-1].strip().capitalize() for a in all_a]
  subj = ['Технология' if ('Направление' in s) else s for s in subj]
  hrefs = [a.get('href') for a in all_a]
  data.append(pd.DataFrame({'year': year, 'subj': subj, 'href': hrefs}))


# таблица ссылок
for year in direct_links:
  r = requests.get(f"https://reg.olimpiada.ru/city-olymp/public/winners/public.html?year={year-1}")
  page = BeautifulSoup(r.text, 'html.parser')
  table = page.find('table')

  rows = table.find_all('tr')
  for row in rows:
      cols = row.find_all('td')
      subj = cols[0].text.split('.')[-1].strip().capitalize() if cols else []
      all_a = [col.find('a') for col in cols[1:]] if cols else []
      hrefs = ["https://reg.olimpiada.ru/city-olymp/public/winners/" + a.get('href') for a in all_a]
      data.append(pd.DataFrame({'year': year, 'subj': subj, 'href': hrefs}))


doc_links = pd.concat(data)

In [ ]:
doc_links

,year,subj,href
0,2023,Английский язык,https://reg.cpm.moscow/register/russia-olympia...
1,2023,Астрономия,https://reg.cpm.moscow/register/russia-olympia...
2,2023,Биология,https://reg.cpm.moscow/register/russia-olympia...
3,2023,География,https://reg.cpm.moscow/register/russia-olympia...
4,2023,Информатика,https://reg.cpm.moscow/register/russia-olympia...
...,...,...,...
1,2015,Экология,https://reg.olimpiada.ru/city-olymp/public/win...
2,2015,Экология,https://reg.olimpiada.ru/city-olymp/public/win...
0,2015,Экономика,https://reg.olimpiada.ru/city-olymp/public/win...
1,2015,Экономика,https://reg.olimpiada.ru/city-olymp/public/win...


In [ ]:
TABLES = []

for index, df_row in doc_links.iterrows():
    r = requests.get(df_row.href)
    page = BeautifulSoup(r.text, 'html.parser')
    table = page.find('table')
    rows = table.find_all('tr') if table else []

    classes = []
    names = []
    results = []
    scores = []

    for row in rows[1:]:
      cols = [c.text.strip() for c in row.find_all('td')]
      if df_row.year >= 2020:
        classes.append(cols[2])
        names.append(cols[1].replace('. ', '.'))
      else:
        classes.append(cols[3])
        names.append(cols[0] + ' ' + cols[1] + cols[2])
      results.append(cols[-1])
      scores.append(cols[-2])

    TABLES.append(pd.DataFrame({'год': df_row.year, 'предмет': df_row.subj,
                                'класс': classes, 'фио': names,
                                'результат': results, 'баллы': scores}))


    save_csv(TABLES, "г. Москва", "moscow.csv")

### Московская область

In [ ]:
links = {2023: "https://olympmo.ru/vos-reg-2022-2023.html.html",
         2022: "https://olympmo.ru/vos-reg-2021-2022.html",
         2021: "https://olympmo.ru/vos-reg-2020-2021.html",
         2020: "https://olympmo.ru/regionalnyij-etap-v-2019-2020.html",
         2019: "https://olympmo.ru/regionalnyij-etap-2018-2019.html"}

In [ ]:
data = []
for year, link in links.items():
  r = requests.get(link)
  page = BeautifulSoup(r.text, 'html.parser')
  table = page.find_all('table')[-1]
  table_body = table.find('tbody')

  rows = table_body.find_all('tr')
  for row in rows:
      cols = row.find_all('td')
      subj = cols[0].text.split('.')[-1].strip() if cols else []
      all_a = cols[-1].find_all('a') if cols else []
      href_names = [a.text.lower() for a in all_a]
      hrefs = [a.get('href') for a in all_a]
      data.append(pd.DataFrame({'year': year, 'subj': subj,
                                'href_name': href_names, 'href': hrefs}))

doc_links = pd.concat(data)

In [ ]:
doc_links['href_name'] = doc_links.href_name.apply(lambda x:
                                                x.split(',')[-1].strip()
                                                if ',' in x and
                                                  x.split(',')[0] in ('кд', 'тт')
                                                else x)
doc_links = doc_links.query("not (year == 2019 and href_name != 'результаты')")
doc_links['class'] = doc_links.href_name.apply(lambda x:
                                                int(x.split()[0])
                                                if ' ' in x and
                                                  x.split()[-1] == 'класс' and
                                                  x.split()[0].isdigit()
                                                else None)

In [ ]:
doc_links

,year,subj,href_name,href,class
0,2023,Французский язык,9-11 класс,news_img/2023/vos/reg/results_with_points/itog...,NaN
0,2023,Искусство (МХК),"9, 10, 11 класс",news_img/2023/vos/reg/results_with_points/itog...,NaN
0,2023,Немецкий язык,9-11 класс,news_img/2023/vos/reg/results_with_points/itog...,NaN
0,2023,Экономика,"9, 10, 11 класс",news_img/2023/vos/reg/results_with_points/itog...,NaN
0,2023,Испанский язык,9-11 класс,news_img/2023/vos/reg/results_with_points/itog...,NaN
...,...,...,...,...,...
0,2019,Немецкий язык,результаты,news_img/old/files/germ.pdf,NaN
0,2019,Испанский язык,результаты,news_img/old/files/span.pdf,NaN
0,2019,Итальянский язык,результаты,news_img/old/files/ital.pdf,NaN
0,2019,Китайский язык,результаты,news_img/old/files/chin.pdf,NaN


In [ ]:
for index, row in doc_links.iterrows():
    name = row.href.split("/")[-2] + ".pdf" if row.href[:5] == 'https' else row.href.split("/")[-1]
    href = row.href if row.href[:5] == 'https' else f"https://olympmo.ru/{row.href}"
    try:
      func_timeout(10, wget.download, args=(href, name))
    except:
      continue

    if name.split(".")[-1].lower() == 'pdf':
      table = extract_table_pdf(name)
    else:
      table = extract_table_docx(name)

    if row['year'] == 2019 or (row['year'] == 2020 and row.subj in ['Немецкий язык', 'Испанский язык', 'Итальянский язык', 'Китайский язык']):
      table = pd.concat([table, pd.DataFrame(table.columns)])
      table.columns = range(table.columns.size)
      table.rename(columns={1: 'Участник', 3: 'класс', 5: 'результат', 4: 'баллы'}, inplace=True)
      table['результат'] = table['результат'].apply(
          lambda x: x.split()[-1].lower() if ' ' in str(x) else x) if 'результат' in table.columns else None
      table['баллы'] = table['баллы'].apply(
          lambda x: x.split()[0] if ' ' in str(x) else x) if 'баллы' in table.columns else None
    else:
      table.columns = table.iloc[0]
      table.drop(0, axis=0, inplace=True)

    if row['year'] == 2021 or (
        row['year'] == 2020 and row.subj not in ['Немецкий язык', 'Испанский язык', 'Итальянский язык', 'Китайский язык']):
      if row.subj == "Основы безопасности жизнедеятельности":
        table.columns = table.iloc[2]
        table.drop(1, axis=0, inplace=True)
        table.drop(2, axis=0, inplace=True)
        table.drop(3, axis=0, inplace=True)
      table.rename(columns={"ФИО": 'фио', 'Класс': 'класс', 'Статус': 'результат', 'Балл': 'баллы'}, inplace=True)
      table['результат'] = table['результат'].apply(lambda x: x.lower() if isinstance(x, str) else x)  if 'результат' in table.columns else None
      table['класс'] = table['класс'].apply(lambda x: x.lower() if isinstance(x, str) else x) if 'класс' in table.columns else row['class']
      table['фио'] = table['фио'].apply(
          lambda x: x.split()[:-1] if ' ' in str(x) and x.split()[-1].isdigit() else x) if 'фио' in table.columns else None
      if row.subj == "Астрономия":
        table.drop(1, axis=0, inplace=True)
    if 'Участник' in table.columns:
      table['Участник'] = table['Участник'].apply(
          lambda x: x.split()[1:] if ' ' in str(x) and x.split()[0].isdigit() else x)
    if 'Класс обучения' in table.columns and 'Класс участия' in table.columns:
      table.drop('Класс обучения', axis=1, inplace=True)
      table['Класс участия'] = table['Класс участия'].apply(lambda x: x.split()[0] if ' ' in str(x) and x.split()[0].isdigit() else x)
      table.rename(columns={"Класс участия": 'класс'}, inplace=True)
    if not any(['класс' in str(x).lower() for x in table.columns]):
      table['класс'] = row['class']

    table.rename(columns={"Участник": 'фио', 'Статус участника': 'результат', 'Итоговый балл': 'баллы'}, inplace=True)
    table['год'] = row['year']
    table['предмет'] = row['subj']
    try:
      df = table[['год', 'предмет', 'фио', 'класс', 'результат', 'баллы']]
    except:
      df = table
    TABLES.append(df.loc[:,~df.columns.duplicated()])


save_csv(TABLES, "Московская область", "mos_obl.csv")